In [2]:
import numpy as np

## Interception

In [17]:
def interception(Potential_Evaporation, Precipitation, Temp, Interceptionstorage, Interceptionstoragecapacity, Temp_Thresh):
   
#     if interception capacity is 0
    if Temp > Temp_Thresh:
        if Interceptionstoragecapacity == 0:
            Effective_Precipitation = Precipitation
            Interception_Evaporation == 0
            Interceptionstorage = 0
        elif Precipitation > 0: #if the temperature is higher than freezing temp, precipitation falls as rain
            Interceptionstorage = Interceptionstorage + Precipitation
            # the excess precipitation will leave the reservoir directly
            Effective_Precipitation = max(0, Interceptionstorage - Interceptionstoragecapacity)
            #change in storage if Effective_Precipitation > 0
            if Effective_Precipitation > 0:
                Interceptionstorage = Interceptionstoragecapacity
            Interception_Evaporation = min(Interceptionstorage, Potential_Evaporation / 2)
            if Interception_Evaporation > 0.0:
                #change in storage
                Interceptionstorage = Interceptionstorage - Interception_Evaporation
        else:
            # if it does not rain, no excess water leaves storage
            Effective_Precipitation = 0
            # the Interception Evporation will be either the amount stored or 50% the potential evaporation
            Interception_Evaporation = min(Interceptionstorage, Potential_Evaporation /2)
            # the amount stored in the Interception Reservoir will decrease because of evaporation
            if Interception_Evaporation > 0:
                #change in storage
                Interceptionstorage = Interceptionstorage - Interception_Evaporation
    else:
        # snow accumulates
        # evporation is 0 and no effective precipitation is released
        Interception_Evaporation = 0.0
        Effective_Precipitation = 0.0
        #Interceptionstorage = Interceptionstorage #amount stored does not change??! 
        
    return Effective_Precipitation, Interception_Evaporation, Interceptionstorage

# Snow

In [18]:
def snow(Area_Glacier, Precipitation, Temp, Snowstorage, Meltfactor, Mm, Temp_Thresh):
    
    if Temp > Temp_Thresh:
        # if temperature higher than freezing temperature, melting takes place
        Melt = Meltfactor * Mm * ( (Temp - Temp_Thresh) / Mm + np.log(1 + np.exp(- (Temp - Temp_Thresh) / Mm) ))
        # it can only melt as much as it is stored in snow storage
        Melt_Snow = min(Melt, Snowstorage)
        Melt_Glacier = Melt
        # the total melt is the combination of snow and glacier melt and the areal extent
        Melt_Total = Melt_Snow * (1 - Area_Glacier) + Melt_Glacier * Area_Glacier
        # the amount of snow stored decreases by amount melted
        Snowstorage = Snowstorage - Melt_Snow
        Melt_Glacier = Melt * Area_Glacier - Melt_Snow * Area_Glacier
    else:
        # the amount of snow stored increases by Precipitation
        Snowstorage = Snowstorage + Precipitation
        # no snow melts
        Melt_Total = 0
        Melt_Glacier = 0
        
    return Melt_Glacier, Melt_Total, Snowstorage

## Soil Hillslope

In [19]:
def soilstorage(Effective_Precipitation, Interception_Evaporation, Potential_Evaporation, Soilstorage, beta, Ce,
               Ratio_Pref, Soilstoragecapacity):
    
    if Effective_Precipitation > 0:
        Ratio_Soil = 1 - (1 - (Soilstorage/Soilstoragecapacity))**beta
        if Ratio_Soil < 0:
            Ratio_Soil = 0
        elif Ratio_Soil > 1:
            Ratio_Soil = 1
        # part of the water enters the soil, it cannot exceed the soil storage capacity
        Unused_Capacity = Soilstoragecapacity - Soilstorage
        Q_Soil = (1 - Ratio_Soil) * Effective_Precipitation
        if Unused_Capacity > Q_Soil:
            Soilstorage = Soilstorage + Q_Soil
        else:
            Q_Soil = Unused_Capacity
            Soilstorage = Soilstoragecapacity
        Overlandflow = (Effective_Precipitation - Q_Soil) * Ratio_Pref
        # or flows into the groundwater
        Preferentialflow = (Effective_Precipitation - Q_Soil) * (1 - Ratio_Pref)
    else:
        # if it does not rain no overland flow occurs
        Overlandflow = 0.0
        Preferentialflow = 0.0
    # Transpiration in soil, only the part that not evaporated in interception reservoir can evaporate
    Potential_Soilevaporation = max(Potential_Evaporation - Interception_Evaporation, 0)
    # transpiration can maximum be the amount stored in soil, or a percentage of potential evaporation
    Soil_Evaporation = Potential_Soilevaporation * min(Soilstorage / (Soilstoragecapacity * Ce), 1)
    Soil_Evaporation = min(Soilstorage, Soil_Evaporation)
    Soilstorage = Soilstorage - Soil_Evaporation
    
    return Overlandflow, Preferentialflow, Soil_Evaporation, Soilstorage

# Soil Riparian

In [20]:
def ripariansoilstorage(Effective_Precipitation, Interception_Evaporation, Potential_Evaporation, 
                        Riparian_Discharge, Soilstorage, beta, Ce, Drainagecapacity, Soilstoragecapacity):
    
    Ratio_Soil = 1 - (1 - (Soilstorage/Soilstoragecapacity))**beta

    Unused_Capacity = Soilstoragecapacity - Soilstorage
    Q_Soil = (1 - Ratio_Soil) * (Effective_Precipitation + Riparian_Discharge)
    if Unused_Capacity > Q_Soil:
        Soilstorage = Soilstorage + Q_Soil
    else:
        Q_Soil = Unused_Capacity
        Soilstorage = Soilstoragecapacity    
    # the other part does not enter the soil but flows into the fast reservoir
    Overlandflow = (Effective_Precipitation + Riparian_Discharge - Q_Soil)
    # Transpiration in soil, only the part that not evaporated in interception reservoir can evaporate
    Potential_Soilevaporation = max(Potential_Evaporation - Interception_Evaporation, 0)
    # transpiration can maximum be the amount stored in soil, or a percentage of potential evaporation
    Soil_Evaporation = Potential_Soilevaporation * min(Soilstorage / (Soilstoragecapacity * Ce), 1)
    Soil_Evaporation = min(Soilstorage, Soil_Evaporation)
    Soilstorage = Soilstorage - Soil_Evaporation
    # Part of the water stored in soil will drain to a maximum capacity, which is routed into the fast response reservoir
    if Drainagecapacity > 0:
        Fastdrainage = (Soilstorage / Soilstoragecapacity) * Drainagecapacity
        Soilstorage = Soilstorage - Fastdrainage
        Overlandflow = Overlandflow + Fastdrainage
        
    return Overlandflow, Soil_Evaporation, Soilstorage

## Fast Reservoir

In [21]:
def faststorage(Overlandflow, Faststorage, Kf):
    
    # the fast storage increases with the overland flow
    Faststorage = Faststorage + Overlandflow
    # a part of the fast storage gets redirected into discharge depending on the reservoir constant (linear response)
    if Kf * Faststorage < Faststorage:
        Fast_Discharge = Kf * Faststorage
        Faststorage = Faststorage - Fast_Discharge
    else:
        Fast_Discharge = Faststorage
        Faststorage = 0    
        
    return Fast_Discharge, Faststorage

## Slow reservoir

In [22]:
def slowstorage(GWflow, Slowstorage, Area_Riparian, Ks, Ratio_Riparian):
    
    Slowstorage = Slowstorage + GWflow
    Slow_Discharge = Ks * Slowstorage * (1 - Ratio_Riparian)
    # the riparian discharge is the areal percentage of the total possible riparian discharge
    Riparian_Discharge = Ks * Slowstorage * Ratio_Riparian * Area_Riparian
    Slowstorage = Slowstorage - Slow_Discharge - Riparian_Discharge 
    
    return Riparian_Discharge, Slow_Discharge, Slowstorage